> Credits: [aws-samples url](https://github.com/aws-samples/function-calling-using-amazon-bedrock-anthropic-claude-3/blob/main/main.py)

### Function calling using Amazon Bedrock with Anthropic Claude 3
> Notebook Source [aws-samples url](https://github.com/aws-samples/function-calling-using-amazon-bedrock-anthropic-claude-3/blob/main/main.py)

In [ ]:
import yfinance as yf
from typing import Union
import sys


ticker_tool_description = {
    "toolSpec": {
        "name": "get_stock_price",
        "description": "Retrieves the current stock price for a given ticker symbol, and the currency that its being traded. The ticker symbol must be a valid symbol for a publicly traded company on a major US stock exchange like NYSE or NASDAQ. The tool will return the latest trade price in USD. It should be used when the user asks about the current or most recent price of a specific stock. It will not provide any other information about the stock or company.",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "The ticker symbol of the company. e.g AAPL for Apple Inc.",
                    }
                },
                "required": ["ticker"],
            },
        },
    }
}


def get_stock_price(ticker: str) -> Union[float, str]:
    """
    Retrieves the current stock price for the given ticker symbol.
    :param ticker: The ticker symbol of the company.
    :return: A tuple containing the current stock price and the currency code.
    """    
    stock = yf.Ticker(ticker)
    # print(f"Stock: {stock}")
    
    info = stock.fast_info
    # print(f"Info: {info}")

    hist = stock.history(period="1d")
    # print(f"Historical Data: {hist}")

    current_price = hist["Close"].iloc[0]
    # print(f"Current Price: {current_price}")
    
    return float(current_price), info["currency"]


def parse_and_run_get_stock_price(tool_use_id, input):
    """
    Parses the given tool inputs, to pass to the tool, and return the tool result formatted for Bedrock Converse API
    :param tool_use_id: the id of the tool use
    :param input: the tool input { ticker: str }
    :return: The tool result formatted for Bedrock Converse API
    """
    ticker = input.get("ticker", {})
    if ticker:
        price, currency = get_stock_price(ticker)
        tool_result = {
            "toolUseId": tool_use_id,
            "content": [
                {
                    "json": {
                        "ticker": ticker,
                        "price": price,
                        "currency": currency,
                    }
                }
            ],
        }
        tool_result_message = {"role": "user", "content": [{"toolResult": tool_result}]}

    return tool_result_message


In [ ]:
from currency_converter import CurrencyConverter

# Tool description to pass the LLM to learn about which tool he may use each turn
currency_converter_tool_description = {
    "toolSpec": {
        "name": "convert_currency",
        "description": "Converts a given amount from one currency to another. The user should provide the amount, the source currency, and the target currency. The tool will return the converted amount in the target currency. It should be used when the user needs to convert one currency to another, such as when purchasing a currency pair or converting cash into a different currency.",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "amount": {
                        "type": "number",
                        "description": "The amount of the source currency to be converted.",
                    },
                    "source_currency": {
                        "type": "string",
                        "description": "The currency of the amount provided. e.g USD for US Dollars.",
                    },
                    "target_currency": {
                        "type": "string",
                        "description": "The currency to convert the amount to. e.g EUR for Euros.",
                    },
                },
                "required": ["amount", "source_currency", "target_currency"],
            },
        },
    }
}


def parse_and_run_convert_currency(tool_use_id, input):
    """
    Parses the given tool inputs, to pass to the tool, and return the tool result formatted for Bedrock Converse API
    :param tool_use_id: The tool use id to uniquely identify the tool use
    :param input: The tool inputs, {amount: float, source_currency: str, target_currency: str}
    :return: The tool result formatted for Bedrock Converse API
    """
    amount = input.get("amount", {})
    source_currency = input.get("source_currency", {})
    target_currency = input.get("target_currency", {})
    converted_currency = convert_currency(amount, source_currency, target_currency)
    tool_result = {
        "toolUseId": tool_use_id,
        "content": [
            {
                "json": {
                    "converted_currency": converted_currency
                }
            }
        ]
    }
    tool_result_message = {
        "role": "user",
        "content": [{"toolResult": tool_result}],
    }
    return tool_result_message



def convert_currency(amount: float, source_currency: str, target_currency: str) -> float:
    """
    Convert the given amount from the source currency to the target currency.
    :param amount: The amount of the source currency.
    :param source_currency: The currency of the amount provided.
    :param target_currency: The currency to convert the amount to.
    :return: The converted amount in the target currency.
    """
    c = CurrencyConverter()
    converted_amount = c.convert(amount, source_currency, target_currency)
    return float(converted_amount)

In [ ]:
import boto3
from typing import Union


bedrock_client = boto3.client("bedrock-runtime", region_name="us-west-2")

# Bedrock model selected
# See supported models:
# https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html#conversation-inference-supported-models-features
modelId = "anthropic.claude-3-sonnet-20240229-v1:0" # Claude Sonnet 3
# modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0" # Claude Sonnet 3.5
# modelId = "cohere.command-r-v1:0" # Command R
# modelId = "cohere.command-r-plus-v1:0" # Command R+
# modelId = "mistral.mistral-large-2407-v1:0" # Mistral Large
# modelId = "meta.llama3-1-70b-instruct-v1:0" # Meta LLAMA3.1 70B

# Converse API inferense parameters
kwargs = {
    "temperature": 0,
    "maxTokens": 2048,
    "topP": 0,
}


system_text_promot = """you are a stock market bot, that provides accurate ticker prices at any currency.
use your tools to get stock price, and covert to another currency when asked.
You answer only questions on companies ticker value and currency.
"""


def generate_text(messages) -> Union[str, list[str], list[dict[str, any]]]:
    """
    Generate the Amazon Bedrock model response for a given input text, and return the response of each turn
    :param messages: list of message dicts from the user
    :return: stop reason, list of the tools requested, and the output messages from the model
    """
    
    system_prompt = [{"text": system_text_promot}]

    # Using Amazon Bedrock converse API
    # https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html
    response = bedrock_client.converse(
        modelId=modelId,
        messages=messages,
        system=system_prompt,
        toolConfig={
            "tools": [ticker_tool_description, currency_converter_tool_description]
        },
        inferenceConfig=kwargs,
    )

    output_message = response.get("output", {}).get("message", {})
    stop_reason = response.get("stopReason")
    tools_requested = []
    # Only if a tools should be used, send the tool to use, and append the messages for the user and assistant to build a conversation
    if stop_reason == "tool_use":
        tools_requested = (
            response.get("output", {}).get("message", {}).get("content", {})
        )
        messages.append(output_message)
        return stop_reason, tools_requested, messages

    return stop_reason, tools_requested, output_message

In [ ]:
import argparse
import json
from loguru import logger


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--input', type=str, required=False, help='Input text for the LLM', default="What is the current stock price of amazon stock in pounds?")
    return parser.parse_known_args()


def build_message(input_text:str) -> list[dict]:
    """
    Build the message to send to the LLM from the input test
    :param input_text: the input text to send to the LLM
    :return: a list containing containing the role, content and user input as text
    """
    return [{"role": "user", "content": [{"text": input_text}]}]


def main(question):
    logger.info("Starting")
    msg = build_message(question)
    logger.info(f"input text: {question}")
    
    stop_reason: str = None
    answer: str = None

    # Run until the model end_turn
    while stop_reason != 'end_turn':
        stop_reason, tools_requested, messages = generate_text(msg)
        logger.debug(f"stop reason is {stop_reason}, continue work till final answer")
        
        # Amazon Bedrock LLM ended turn and responded the final answer
        if stop_reason == 'end_turn':
            logger.info("The question asked the LLM ended turn and this is the answer")
            answers = messages.get("content", {})
            # itterate over the returned answers from Amazon Bedrock LLM
            answers_text = [a.get('text', '\n') for a in answers]
            answer = ''.join(answers_text)
            break
        
        if stop_reason == 'tool_use':
            # find from the content returned form tools_requested the tool to use
            for content in tools_requested:
                if 'toolUse' in content:
                    tool_use_id = content.get('toolUse', {}).get('toolUseId')
                    tool_use_name = content.get('toolUse', {}).get('name')
                    tool_use_input = content.get('toolUse', {}).get('input')
                    logger.info(f"tool use id is {tool_use_id}, tool use name is {tool_use_name}")
                    # stock price tool
                    if tool_use_name == 'get_stock_price':
                        message = parse_and_run_get_stock_price(tool_use_id, tool_use_input)
                        messages.append(message)
                    
                    # currency conversion tool
                    if tool_use_name == 'convert_currency':
                        message = parse_and_run_convert_currency(tool_use_id, tool_use_input)
                        messages.append(message)
                
                # See the messages appended that are being built for the LLM, this will allow the Bedrock LLM to provide the final answer.
                logger.debug(f"messages:\n{json.dumps(messages)}")

        else:
            # Stop reasons can be: 'end_turn'|'tool_use'|'max_tokens'|'stop_sequence'|'guardrail_intervened'|'content_filtered'
            # This code sample only covers end_turn, and tool_use, you may need to implement additional code to cover all the rest of the responses.
            logger.warning(f"llm didn't end_turn, or asked to use a tool, he asked to {stop_reason}")
            return
        
    # Printing the final reponse from the model
    logger.info(answer)

### Posible Error:
> Si reciben error 'AMZNM : No data found, symbol may be delisted' con la acción que estén buscando, instalen nuevamente [yfinance](https://pypi.org/project/yfinance/)

In [ ]:
main("What is the latest stock price of Amazon stock in pounds?")